In [32]:
import pandas as pd
import numpy as np
import prepare as p

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
                                  
import graphviz
from graphviz import Graph

from scipy import stats

In [33]:
df = pd.read_csv('data/healthcare-dataset-stroke-data.csv')

In [35]:
df, modeling_df = p.prep_data(df)

In [36]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_range
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1,65-70
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,30.640331,never smoked,1,60-65
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1,75-80
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1,45-50
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1,75-80


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5109 entries, 0 to 5108
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5109 non-null   object 
 1   gender             5109 non-null   object 
 2   age                5109 non-null   float64
 3   hypertension       5109 non-null   object 
 4   heart_disease      5109 non-null   object 
 5   ever_married       5109 non-null   object 
 6   work_type          5109 non-null   object 
 7   residence_type     5109 non-null   object 
 8   avg_glucose_level  5109 non-null   float64
 9   bmi                5109 non-null   float64
 10  smoking_status     5109 non-null   object 
 11  stroke             5109 non-null   object 
 12  age_range          5109 non-null   object 
dtypes: float64(3), object(10)
memory usage: 519.0+ KB
